In [ ]:
import numpy as np # linear algebra
import pandas as pd 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from subprocess import check_output
print(check_output(["ls","CleanData/data_clean.csv"]).decode("utf8"))

In [ ]:
#Load Chicago Taxi trip data
%time
# We'll load some important columns only
df = pd.read_csv('CleanData/data_clean.csv',
                 usecols=['trip_start_timestamp', 'trip_end_timestamp', 'fare', 'pickup_latitude', 'pickup_longitude',
                          'dropoff_latitude', 'dropoff_longitude'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
from bokeh.plotting import figure, output_notebook, show # bokeh plotting library
# We'll show the plots in the cells of this notebook
output_notebook()

In [ ]:
print(np.min(df['pickup_latitude']), np.min(df['pickup_longitude']))
print(np.max(df['pickup_latitude']), np.max(df['pickup_longitude']))

print(np.min(df['dropoff_latitude']), np.min(df['dropoff_longitude']))
print(np.max(df['dropoff_latitude']), np.max(df['dropoff_longitude']))

In [ ]:
Chicago = x_range, y_range = ((41.68, 42.00), (-87.55, -87.92))

In [ ]:
plot_width = int(750)
plot_height = int(plot_width//1.2)

def base_plot(tools='pan, wheel_zoom, reset', plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
              x_range=x_range, y_range=y_range, outline_line_color=None,
              min_border=0, min_border_left=0, min_border_right=0,
              min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

options = dict(line_color=None, fill_color='blue', size=5)

In [ ]:
%%time
# let's plot 10k sample pickup
samples = df.sample(n=10000)
p = base_plot()

p.circle(x=samples['pickup_latitude'], y=samples['pickup_longitude'], **options)
show(p)

In [ ]:
import datashader as ds
from datashader import transfer_functions as tr_fns
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:2]

In [ ]:
%%time
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude')
img = tr_fns.shade(agg, cmap=["white", 'darkblue'], how='linear')

img

In [ ]:
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

background = "black"
export = partial(export_image, export_path="export", background=background)
cm = partial(colormap_select, reverse=(background=="black"))

def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude')
    img = tr_fns.shade(agg, cmap=Hot, how='eq_hist')
    return tr_fns.dynspread(img, threshold=0.5, max_px=4)

p = base_plot(background_fill_color=background)
export(create_image(*Chicago), "Chicago_HOT")
InteractiveImage(p, create_image)

In [ ]:
from functools import partial

def create_image90(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude')
    img = tr_fns.shade(agg.where(agg > np.percentile(agg, 90)), cmap=inferno, how='eq_hist')
    return tr_fns.dynspread(img, threshold=0.3, max_px=4)
    
p = base_plot()
export(create_image(*Chicago), "Chicago_90th")
InteractiveImage(p, create_image90)

In [ ]:
def merged_images(x_range, y_range, w=plot_width, h=plot_height, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    picks = cvs.points(df, 'pickup_longitude', 'pickup_latitude')
    drops = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude')
    more_drops = tr_fns.shade(drops.where(drops > picks), cmap=["darkblue", 'cornflowerblue'], how=how)
    more_picks = tr_fns.shade(drops.where(picks > drops), cmap=["darkred", 'orangered'], how=how)
    img = tr_fns.stack(more_picks, more_drops)
    return tr_fns.dynspread(img, threshold=0.3, max_px=4)

p = base_plot(background_fill_color=background)
export(merged_images(*Chicago), "Chicago_pickups_vs_drops")
InteractiveImage(p, merged_images)